<img align="right" style="display:inline;" width="400" src="https://camo.githubusercontent.com/c288679ac2172d1804d8c73e0bb79e066f57e358/68747470733a2f2f676973742e6769746875622e636f6d2f646576696e2d7065746572736f686e2f66343234643966623535373961393635303763373039613336643438376632342f7261772f343936333164333739623662363364613566313833383962346661363061643433363465373764352f726973656c61622d61742d75632d6265726b656c65792e6a7067">

<br><br><br><br><br><br><br><br><br>
<b><font size="7">Modin (Pandas on Ray)</font></b>

<h3>Accelerate your pandas workflows by changing one line of code</h3>

<br><br>

##### Devin Petersohn

# An anecdote to get started
My background: Genomics and Computational Biology Data Science
<br><br>

**Comments from a Data Scientist who runs production genomics workloads:** 
- "Data is too large to use in pandas (10's of GB to TB)"
- "I want to interact with my data"
- "I end up using Big Data tools to trim the data down and use pandas to analyze it"

<h1><center>"Why can't we use the same tools for Kilo- and Megabyte-scale data as we do for Terabyte-scale data (and vice versa)?"</center></h1>

<font color="navy"><h1>Current Data Science Landscape</h1></font>
<br><br>

![](mbTools.png)

<font color="navy"><h1>Current Data Science Landscape</h1></font>
<br><br>

![](tbTools.png)

<font color="navy"><h1>Current Data Science Landscape</h1></font>
<br><br>
![](toolsNoArrow.png)

<font color="navy"><h1>Current Data Science Landscape</h1></font>
<br><br>
![](toolsFinal.png)

<font color="navy"><h1>Current Data Science Landscape (1TB+)</h1></font>
<br><br>

- New Frameworks = New APIs
    - Many also support SQL - This is good!
    - However, many pandas operations not covered by SQL (e.g. `iloc`)
- Expose distributed computing concepts to users
    - Partitioning and shuffling
    - Tuning crucial for performance
- Many are too heavyweight for good performance at smaller scale
- Optimized for batching, not necessarily optimized for 2D matrix operations

<center><h1>What is it about extracting value from data that requires expertise in distributed computing?</h1></center>
<br><br><br><br>
<center><h3>Nothing!</h3></center>

<center><h1>With Modin, we are trying to bridge the gap between analytics at MBs and TBs+</h1></center>

# Modin (모든)

### Accelerate your pandas workflows by changing one line of code
<br><br>

In [1]:
# import pandas as pd
import modin.pandas as pd

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2018-11-04_14-29-56_67462/logs.
Waiting for redis server at 127.0.0.1:53424 to respond...
Waiting for redis server at 127.0.0.1:45458 to respond...
Starting the Plasma object store with 13.743895347 GB memory using /tmp.


<center><h4>This notebook was run on a 2013 iMac with 4-cores and 32GB RAM</h4></center>

In [2]:
import numpy as np

# Build a 2D numpy array filled with random integers
frame_data = np.random.randint(0, 100, size=(2**16, 2**4))
# Put the new random data into a dataframe
df = pd.DataFrame(frame_data)
# Add a prefix to each column for simplicity
df = df.add_prefix("col_")
type(df)

modin.pandas.dataframe.DataFrame

In [3]:
# Print the first 10 lines of the DataFrame
df.head(10)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15
0,8,35,40,76,53,12,13,42,31,12,50,14,49,19,94,30
1,56,48,51,95,60,6,95,63,24,85,35,47,55,75,84,7
2,15,43,53,34,6,79,71,98,52,95,11,98,72,2,89,10
3,53,93,46,50,61,94,33,44,84,50,74,66,42,69,73,2
4,81,53,70,99,72,23,4,96,2,15,57,7,28,17,64,97
5,61,76,65,26,92,8,4,62,2,37,38,4,97,29,86,72
6,73,10,72,46,37,20,29,8,56,33,97,94,76,97,5,75
7,98,86,1,79,2,49,73,70,22,18,70,86,93,55,73,78
8,35,65,34,91,77,12,31,62,11,41,82,99,20,54,17,83
9,36,38,45,6,50,42,16,54,52,1,18,31,56,78,40,76


<center><h1>Modin manages the data partitioning and shuffling, so you can focus on extracting value from your data</h1></center>
<br><br><br><br>
<center><h3>After all, isn't that the purpose of Data Science?</h3></center>

# <center><code>pd.read_csv</code></center>
<br>

In [4]:
import pandas

#### pandas

In [5]:
%%time
# pandas `read_csv`
pandas_csv_data = pandas.read_csv("800MB.csv")

CPU times: user 27.4 s, sys: 3.32 s, total: 30.7 s
Wall time: 30.7 s


#### Modin

In [6]:
%%time
# Modin `read_csv`
csv_data = pd.read_csv("800MB.csv")

CPU times: user 62.5 ms, sys: 5.21 ms, total: 67.8 ms
Wall time: 7.99 s


# <center><code>df.groupby</code></center>
<br><br>

#### pandas

In [7]:
%%time
# pandas `groupby`
_ = pandas_csv_data.groupby(by=pandas_csv_data.col_1).sum()

CPU times: user 6.03 s, sys: 1.75 s, total: 7.78 s
Wall time: 7.76 s


#### Modin

In [8]:
%%time
# Modin `groupby`
results = csv_data.groupby(by=csv_data.col_1).sum()

CPU times: user 2.11 s, sys: 38.2 ms, total: 2.14 s
Wall time: 5.89 s


# <center><code>df.T</code></center>

#### Modin

In [9]:
%%time
# Modin transpose
results_transpose = results.T

CPU times: user 38 µs, sys: 7 µs, total: 45 µs
Wall time: 49.1 µs


In [10]:
results_transpose.tail()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
col_251,510687,532665,531041,518312,514216,512767,517570,522186,512149,518341,...,516180,516238,518686,522299,530929,518363,515769,519990,520853,516718
col_252,506608,532956,527241,527219,517928,512462,518954,518623,510923,523518,...,517377,513713,517771,529987,534633,513980,511530,518929,523025,514076
col_253,507078,530762,535021,519887,520246,515980,516653,524330,510801,520746,...,516373,521599,512115,528876,529613,513792,521189,515032,522286,515301
col_254,506274,527389,534761,516021,517681,515188,517922,522157,514011,515284,...,513980,516344,514599,529063,531735,515256,509626,518098,522041,527306
col_255,511399,527227,531609,517430,521920,509612,518645,525048,510239,517014,...,517766,519192,513554,532277,530755,515562,521128,519129,520482,520019


<center><h3>All partitioning and shuffling is handled for you!</h3></center>

<center><h1>How do we get this speedup?</h1></center>
<br><br>
<center><h3>Modern laptop</h3></center>

<center><img src="multicore_start.png"></center>

<center><h3>pandas on your Laptop</h3></center>

<center><img src="pandas_multicore.png"></center>

<center><h3>Modin on your laptop</h3></center>

<center><img src="modin_multicore.png"></center>

<center><h3>pandas on a large machine</h3></center>

<center><img src="pandas_multicore_lots.png"></center>

<center><h3>Modin on a large machine</h3></center>

<center><img src="modin_multicore_lots.png"></center>

<center><h1>Performance of `read_csv` on various amounts of Data using 144 cores</h1></center>
<br><br>
<center><img src="read_csv_plot.png"></center>

<center><h1>Modin is an early stage, multi-process DataFrame library with an identical API to pandas.</h1></center>

<br><br><br><br>

<center><h3>Cluster support coming soon!</h3></center>

<center><h1>The pandas API is massive!</h1></center>
<br><br><br>

**`pd.DataFrame`**
- 280+ methods

**`pd.Series`**
- 280+ methods

**Other operations (`pd.concat`, etc.)**
- 40+ APIs


<center><h3>Where do you even start?</h3></center>

<center><h1>What do people use in the pandas API?</h1></center>

![](https://docs.google.com/spreadsheets/d/e/2PACX-1vSJAqz2lmMe2yxUEV1BDYYJcb7F_javeq1mwW_uoiqOi8WuXQBnDIBAOkeF_WJ9iOtxuJxgvr_8PzFv/pubchart?oid=108581991&format=image)

<center><h3>We can implement and optimize in the order of popularity!</h3></center>

# Implementing and optimizing the pandas API

### Rank-ordered by popularity

<br><br>

- Currently, we support 71.77% of the pandas API in `modin.pandas`
- This represents >93% of usage based on our study
- `pd.Series` is not yet distributed
    - This will help us optimize many operations (e.g. `df.groupby`)
- `pd.MultiIndex` preliminarily supported

<br><br>

<center><h3>What about the rest?</h3></center>

<center><h1>Defaulting to pandas in Modin</h1></center>

<br><br>

In [11]:
# Covariance not yet implemented, but you can still use it in Modin!
cov_df = df.cov()

/Users/DevinPetersohn/software_builds/modin/modin/pandas/dataframe.py:5045: UserWarning: Defaulting to Pandas implementation
  warnings.warn("Defaulting to Pandas implementation", UserWarning)


In [12]:
# Print the first 5 lines of the result. NOTE: This is a Modin DataFrame
cov_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15
col_0,829.008582,-2.354179,-1.416841,-0.842917,1.940283,-1.943185,-2.303167,1.777532,-0.922861,-2.231256,3.861377,-1.437169,1.478369,0.733189,2.861370,1.597719
col_1,-2.354179,832.693612,3.025415,2.695618,0.052238,6.658226,2.228723,0.043625,0.175632,-0.330016,1.275915,-0.242055,0.835773,-5.747407,4.549550,4.387604
col_2,-1.416841,3.025415,831.740214,-6.414016,-1.145057,0.395215,1.900912,0.593963,0.400486,-1.778687,8.051968,-0.065934,2.736739,-1.564379,5.221482,0.741956
col_3,-0.842917,2.695618,-6.414016,834.878803,-8.021696,-0.736039,-3.000398,2.763483,-7.845078,2.142328,3.651759,-5.780861,-3.490790,-4.883697,-1.184995,-3.102995
col_4,1.940283,0.052238,-1.145057,-8.021696,834.906930,-3.574851,1.214925,0.413699,-2.776199,3.811421,2.769459,0.208708,-2.992184,-1.502127,0.419019,2.621853


<center><h1>Defaulting to pandas in Modin</h1></center>

<br><br>

![](convert_to_pandas.png)

<center><h1>Now you can do almost anything in Modin you normally could do in pandas!</h1></center>

<br><br>



# Modin: Behind the Scenes

![](allNoBox.png)

# Modin: Behind the Scenes

![](pandasAPI.png)

# Modin: Behind the Scenes

![](queryCompiler.png)

# Modin: Behind the Scenes

![](partitionManager.png)

# Modin: Behind the Scenes

![](partitions.png)

# Modin: Behind the Scenes

![](all.png)

# What is Ray?

# Ray: A task parallel, low latency execution framework

<br><br><br><br>

![](ray_architecture_diagram.jpg)

# Ray: A system for parallel and distributed Python that unifies the ML ecosystem.

<br><br><br>

![](ray_overview.jpg)

# Ray

- Ray is open source at https://github.com/ray-project/ray
- Visit Ray's documentation for more: http://ray.readthedocs.io/en/latest/index.html

<center><h1>Demo!</h1></center>

# Conclusion

### Modin: Accelerate your pandas by changing one line of code

<br><br>

- API is identical to pandas
- No distributed computing knowledge required!
- Brief architecture overview
- Introduction to Ray

# Modin moving forward

- Memory management
    - Out of core, or DataFrames exceeding memory
- Query planning (exciting research to be done here!)
- Better partitioning planning
- More API coverage
- Distributed `pd.Series`
- Currently support most recent stable release of pandas API (0.23.4)
    - Continue supporting pandas API changes in the future
- Modin is only 8 months old!

<br><br>

# Interested in contributing? We'd love to have you!

- Email the mailing list (found in the documentation)
- Pick an issue!
- Come find me @ PyData!

# Modin (모든)

### Accelerate your pandas workflows by changing one line of code
<br>

```python
# import pandas as pd
import modin.pandas as pd
```

### Modin is open source at https://github.com/modin-project/modin
### Install with `pip install modin`
### Documentation: https://modin.readthedocs.io/en/latest/

<br>

##### Devin Petersohn: devin@eecs.berkeley.edu
<center><h1>Thank you</h1></center>